In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy.io as sio
import harmonypy as hm
import os

#print the current scanpy version
print(sc.__version__)

In [ ]:
os.chdir("C:/Users/AW/Desktop/EGA-model")
print("当前工作目录:", os.getcwd())

In [ ]:
# filter expression matrices to only include HVGs shared across all datasets
def hvg_selection_and_pooling(exp_paths, n_top_genes = 1000):
    #input n expression matrices paths, output n expression matrices with only the union of the HVGs
    #read adata and find hvgs
    hvg_bools = []
    for d in exp_paths:
        adata = sio.mmread(d)
        adata = adata.toarray()
        print(adata.shape)
        adata = sc.AnnData(X=adata.T, dtype=adata.dtype)

        # Preprocess the data
        sc.pp.normalize_total(adata)
        sc.pp.log1p(adata)
        sc.pp.highly_variable_genes(adata, n_top_genes=n_top_genes)

        #save hvgs
        hvg = adata.var['highly_variable']
        hvg_bools.append(hvg)

    #find union of hvgs
    hvg_union = hvg_bools[0]
    for i in range(1, len(hvg_bools)):
        print(sum(hvg_union), sum(hvg_bools[i]))
        hvg_union = hvg_union | hvg_bools[i]

    print("Number of HVGs: ", hvg_union.sum())

    #filter expression matrices
    filtered_exp_mtxs = []
    for d in exp_paths:
        adata = sio.mmread(d)
        adata = adata.toarray()
        adata = sc.AnnData(X=adata.T, dtype=adata.dtype)

        # Preprocess the data and subset
        sc.pp.normalize_total(adata)
        sc.pp.log1p(adata)
        filtered_exp_mtxs.append(adata[:, hvg_union].X)
    return filtered_exp_mtxs


exp_paths = ["EGA-dataset/data/filtered_expression_matrices/1/matrix.mtx",
            "EGA-dataset/data/filtered_expression_matrices/2/matrix.mtx",
            "EGA-dataset/data/filtered_expression_matrices/3/matrix.mtx",
            "EGA-dataset/data/filtered_expression_matrices/4/matrix.mtx",
            "EGA-dataset/data/filtered_expression_matrices/5/matrix.mtx",
            "EGA-dataset/data/filtered_expression_matrices/6/matrix.mtx",
            "EGA-dataset/data/filtered_expression_matrices/7/matrix.mtx",
            "EGA-dataset/data/filtered_expression_matrices/8/matrix.mtx"]

filtered_mtx = hvg_selection_and_pooling(exp_paths)

for i in range(len(filtered_mtx)):
    out_path = f"EGA-dataset/data/filtered_expression_matrices/{i+1}/hvg_matrix.npy"
    np.save(out_path, filtered_mtx[i].T)
    print(f"Saved: {out_path}")

In [ ]:
################################
#! batch correct using harmony
#! Other batch correction methods can be used here in place of harmony. Furthermore, model can be trained using the hvg matrix and achieve comparable results if the datasets used are similar enough

d = np.load("EGA-dataset/data/filtered_expression_matrices/1/hvg_matrix.npy")
print(d.shape)

d2 = np.load("EGA-dataset/data/filtered_expression_matrices/2/hvg_matrix.npy")
print(d2.shape)

d3 = np.load("EGA-dataset/data/filtered_expression_matrices/3/hvg_matrix.npy")
print(d3.shape)

d4 = np.load("EGA-dataset/data/filtered_expression_matrices/4/hvg_matrix.npy")
print(d4.shape)

d5 = np.load("EGA-dataset/data/filtered_expression_matrices/5/hvg_matrix.npy")
print(d5.shape)

d6 = np.load("EGA-dataset/data/filtered_expression_matrices/6/hvg_matrix.npy")
print(d6.shape)

d7 = np.load("EGA-dataset/data/filtered_expression_matrices/7/hvg_matrix.npy")
print(d7.shape)

d8 = np.load("EGA-dataset/data/filtered_expression_matrices/8/hvg_matrix.npy")
print(d8.shape)

d = np.concatenate((d.T, d2.T, d3.T, d4.T, d5.T, d6.T, d7.T, d8.T), axis = 0)

data_sizes = [2706, 2744, 1021, 2060, 2161, 2163, 2801, 3242]
batch_labels = np.concatenate((np.zeros(2706), np.ones(2744), np.ones(1021)*2, np.ones(2060)*3,
                               np.ones(2161)*4, np.ones(2163)*5, np.ones(2801)*6, np.ones(3242)*7))
batch_labels = batch_labels.astype(str)
df = pd.DataFrame(batch_labels, columns=["dataset"])

# # Run the Harmony integration algorithm
harmony = hm.run_harmony(d, meta_data=df, vars_use=["dataset"])
harmony_corrected = harmony.Z_corr.T

#split back into datasets
d1 = harmony_corrected[:data_sizes[0]]
d2 = harmony_corrected[data_sizes[0]:data_sizes[0]+data_sizes[1]]
d3 = harmony_corrected[data_sizes[0]+data_sizes[1]:data_sizes[0]+data_sizes[1]+data_sizes[2]]
d4 = harmony_corrected[data_sizes[0]+data_sizes[1]+data_sizes[2]:data_sizes[0]+data_sizes[1]+data_sizes[2]+data_sizes[3]]
d5 = harmony_corrected[data_sizes[0]+data_sizes[1]+data_sizes[2]+data_sizes[3]:data_sizes[0]+data_sizes[1]+data_sizes[2]+data_sizes[3]+data_sizes[4]]
d6 = harmony_corrected[data_sizes[0]+data_sizes[1]+data_sizes[2]+data_sizes[3]+data_sizes[4]:data_sizes[0]+data_sizes[1]+data_sizes[2]+data_sizes[3]+data_sizes[4]+data_sizes[5]]
d7 = harmony_corrected[data_sizes[0]+data_sizes[1]+data_sizes[2]+data_sizes[3]+data_sizes[4]+data_sizes[5]:data_sizes[0]+data_sizes[1]+data_sizes[2]+data_sizes[3]+data_sizes[4]+data_sizes[5]+data_sizes[6]]
d8 = harmony_corrected[data_sizes[0]+data_sizes[1]+data_sizes[2]+data_sizes[3]+data_sizes[4]+data_sizes[5]+data_sizes[6]:]

print(d1.shape, d2.shape, d3.shape, d4.shape, d5.shape, d6.shape, d7.shape, d8.shape)

#save
np.save("EGA-dataset/data/filtered_expression_matrices/1/harmony_matrix.npy", d1.T)
np.save("EGA-dataset/data/filtered_expression_matrices/2/harmony_matrix.npy", d2.T)
np.save("EGA-dataset/data/filtered_expression_matrices/3/harmony_matrix.npy", d3.T)
np.save("EGA-dataset/data/filtered_expression_matrices/4/harmony_matrix.npy", d4.T)
np.save("EGA-dataset/data/filtered_expression_matrices/5/harmony_matrix.npy", d5.T)
np.save("EGA-dataset/data/filtered_expression_matrices/6/harmony_matrix.npy", d6.T)
np.save("EGA-dataset/data/filtered_expression_matrices/7/harmony_matrix.npy", d7.T)
np.save("EGA-dataset/data/filtered_expression_matrices/8/harmony_matrix.npy", d8.T)

In [ ]:
## save file for hvg_union.npy

def get_hvg_union(exp_paths, n_top_genes=1000):
    hvg_bools = []
    for d in exp_paths:
        adata = sio.mmread(d)
        adata = adata.toarray()
        adata = sc.AnnData(X=adata.T, dtype=adata.dtype)

        sc.pp.normalize_total(adata)
        sc.pp.log1p(adata)
        sc.pp.highly_variable_genes(adata, n_top_genes=n_top_genes)

        hvg = adata.var['highly_variable']
        hvg_bools.append(hvg)

    hvg_union = hvg_bools[0]
    for i in range(1, len(hvg_bools)):
        hvg_union = hvg_union | hvg_bools[i]

    return hvg_union

# save hvg_union
hvg_union = get_hvg_union(exp_paths)
np.save("EGA-dataset/data/filtered_expression_matrices/", hvg_union)
print(f"Saved HVG union mask with {hvg_union.sum()} genes")